In [1]:
import numpy as np
import networkx as nx
import random
import torch
from train import Train_Model
#from tqdm import tqdm, trange
from tqdm import tqdm_notebook as tqdm
import gc
from model import *
from utils import *

In [2]:
def pre_sample_for_reddit(walk_times,adj_sparse, train_index, batch_size, save_name, do_walk):
    if do_walk:
        nodes_num = len(train_index)
        walks = torch.zeros(nodes_num, walk_times+1).long().cuda()
        batches = create_batches_forList(train_index, batch_size, True)
        i=0
        degrees = torch.tensor(adj_sparse.sum(1)).view(-1).cuda()
        #print(degrees, (degrees<0).sum())
        candi_node = 0
        for batch in batches:
            print("batch",batch)
            walks[i*batch_size : i*batch_size + len(batch), 0] = torch.tensor(batch).cuda()
            candi_node = sparse_mx_to_torch_sparse_tensor(adj_sparse[batch]).cuda()
            chosen_node = torch.zeros(len(batch), adj_sparse.shape[0]).cuda()
            for id in range(len(batch)):
                chosen_node[id][batch[id]] = 1.
            candi_node = ((- chosen_node + candi_node)> 0.0).float()
            for walk_id in range(walk_times):
                for x in range(candi_node.shape[0]):
                    if candi_node[x].sum()==0:
                        #print('error')
                        candi_node[x][batch[x]] = 1.
                #print(candi_node.shape,candi_node)
                #print(degrees.shape,degrees)
                #print(candi_node.sum(1), (candi_node.sum(1)<=0).sum())
               
                p = candi_node * degrees
                #print(candi_node.shape, candi_node,(candi_node.sum(1)<=0).sum())
                #print(degrees.shape, degrees,(degrees<0).sum())
                #print(p, (p==0).sum())
                #print(p.shape, p,(p.sum(1)<=0).sum())
                #p = p / (p.sum(1).unsqueeze(1))
                #print(p, (p==0).sum())
                #p == 1
                #print(p,(p<0).sum())
                new_node = torch.multinomial(p,1).squeeze(1)
                #m = torch.distributions.categorical.Categorical(p)
                #new_node = m.sample()
                walks[i*batch_size : i*batch_size + len(batch), walk_id+1] = new_node
                for id in range(len(batch)):
                    chosen_node[id][new_node[id]] = 1.
                candi_node = candi_node - chosen_node + sparse_mx_to_torch_sparse_tensor(adj_sparse[new_node.cpu()]).cuda()
                candi_node = (candi_node> 0.0).float()
                
            i+=1
        torch.cuda.empty_cache()
        result1 = np.array(walks.cpu())
        io.savemat(save_name+'.mat',{save_name:result1})
        return walks
    else:
        walks = io.loadmat(save_name+'.mat')
        return torch.tensor(walks[save_name]).cuda()

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
sparse_adj, sparse_adj_train,features, train_feature,labels, train_labels, id_train, id_valid, id_test, num_labels = load_data2('reddit',True)
sparse_adj.setdiag(1.0) 
sparse_adj_train.setdiag(1.0) 

train_index 152410
val_index 23699
test_index 55334
adj torch.Size([232965, 232965])
features torch.Size([232965, 602])


/opt/conda/lib/python3.7/site-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [8]:
for walk in range(1,10):
    for seed in range(10):
        torch.manual_seed(seed)
        random.seed(seed)
        torch.cuda.manual_seed(seed)
        np.random.seed(seed)
        save_name = 'seed='+str(seed)+'_walk='+str(walk)
        print('process...'+'tain_'+save_name)
        pre_sample_for_reddit(walk,sparse_adj_train, id_train, 256, 'tain_'+save_name, True)
        print('process...'+'valid_'+save_name)
        pre_sample_for_reddit(walk,sparse_adj, id_valid, 256, 'valid_'+save_name, True)
        print('process...'+'test_'+save_name)
        pre_sample_for_reddit(walk,sparse_adj, id_test, 256, 'test_'+save_name, True)

process...tain_seed=0_walk=1
batch tensor([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
         28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
         42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
         56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
         70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
         84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
         98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
        112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
        126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
        140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153,
        154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
        168, 169, 170, 171, 1

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':


IndexError: invalid index